In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import lightgbm as lgb

from tsfresh.examples import load_robot_execution_failures
from tsfresh import extract_features, select_features
import optuna

from common import EP
from dfdb import DFDB

import types
import copy

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning:

numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.

Using TensorFlow backend.


In [2]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [3]:
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_colwidth', -1)

In [4]:
df_train = pd.read_pickle('../feats/df_train.pkl')
df_test = pd.read_pickle('../feats/df_test.pkl')

In [5]:
df_train['label'] = df_train['y'].apply(lambda x:  int(x) if x<15 else 15)
group = df_train['season'].values
group[np.where(group==17)[0]] = 1
df_train['group'] = group
df_train = df_train.drop(columns=['season'])

In [6]:
test_X = df_train.drop(columns=['y','index','group','label']).copy()
test_X.index = df_train['index']
test_y = df_train['y'].copy()
test_y.index = df_train['index']
tsfresh_columns = select_features(test_X, test_y).columns.tolist()

In [7]:
original_columns = df_train.columns.drop(['index','y','label','group']).tolist()

In [8]:
#check feature_importances
# df_feature_importances = df_trial.loc[294]['df_feature_importances']
# sorted_columns = EP.evaluate(df_feature_importances, key='average_permutation_weight')
# df_feature_importances.sort_values(by=['average_permutation_weight'], ascending=False).head(100)
# len(sorted_columns)

In [9]:
# mytrial = []
db = DFDB('../trial/xgbm.pkl', auto_commit=False)

In [10]:
df_trial = db.select()
df_trial['kfold'] = df_trial['param'].apply(lambda x: x['kfold'])

In [14]:
df_trial[['datetime','nfeatures', 'kfold', 'remark', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','remark']].loc[248:273]

,datetime,nfeatures,kfold,remark,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,remark
248,2019-05-06 04:48:20.973679,1820,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,1.991750,0.051529,2.054514,0.717275,0.062764,None
249,2019-05-06 05:48:37.256732,22,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,2.009075,0.037867,2.018946,0.661665,0.009871,None
250,2019-05-06 05:48:45.455757,17,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,1.997562,0.026410,1.971074,0.621503,-0.026489,None
251,2019-05-06 05:52:29.282481,22,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,2.009075,0.037867,2.018946,0.661665,0.009871,None
252,2019-05-06 05:52:40.406186,17,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,1.997562,0.026410,1.971074,0.621503,-0.026489,None
253,2019-05-06 06:09:20.216642,120,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,1.995402,0.046402,2.023155,0.673155,0.027753,None
254,2019-05-06 06:09:53.758052,115,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,1.964263,0.038414,2.019078,0.692620,0.054816,None
255,2019-05-06 06:10:27.488974,110,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,1.968276,0.037392,2.001287,0.682020,0.033011,None
256,2019-05-06 06:10:58.635970,105,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,1.987626,0.040535,1.990721,0.654039,0.003095,None
257,2019-05-06 06:11:28.330412,100,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,1.975875,0.039302,1.984727,0.644857,0.008852,None


In [15]:
param_idx = 273
column_idx = 253
db_ = db
df_trial_ = df_trial
mytrial = []
columns = copy.deepcopy(df_trial_.loc[column_idx]['param']['columns'])
param = copy.deepcopy(df_trial_.loc[param_idx]['param'])
# param['algorithm']['init'] = {'logging_level': 'Silent', **param['algorithm']['init']}
param['scaler']['init'] = {}
param['algorithm']['fit'] = {}

selected_columns = revert_rfe(df_train, param, columns, df_test, mytrial, start_columns=[columns[0]], limit=20, remark='start from top1 column')
print(len(selected_columns))
for trial_i in mytrial:
    db_.insert(trial_i)
df_trial_ = db_.select()
df_trial_['kfold'] = df_trial_['param'].apply(lambda x: x['kfold'])

12


In [22]:
selected_columns

['q25_roll_std_100',
 'abs_q25_5',
 'spkt_welch_density__coeff_3',
 'abs_q75_7',
 'spkt_welch_densitycoeff_2',
 "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
 'agg_autocorrelationf_agg_"mean"__maxlag_40',
 "number_peaks{'n': 1}",
 '5000peak_peak_amp_max_',
 'abs_q95_3',
 'spkt_welch_density__coeff_89',
 'abs_q05_2']

In [12]:
df_trial[df_trial['remark']=='start from top1 column'][['datetime','nfeatures', 'kfold', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,nfeatures,kfold,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
1300,2019-05-22 07:45:45.971968,1,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}",2.058100,0.000010,2.069261,0.000047,0.011161
1301,2019-05-22 07:45:46.968994,2,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}",2.054077,0.000014,2.069573,0.000035,0.015496
1302,2019-05-22 07:45:48.007513,2,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}",2.063218,0.000056,2.080734,0.000008,0.017516
1303,2019-05-22 07:45:48.923666,2,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}",2.056661,0.000004,2.071492,0.000076,0.014831
1304,2019-05-22 07:45:49.910746,2,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}",2.052949,0.000007,2.066940,0.000029,0.013990
1305,2019-05-22 07:45:51.038370,3,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}",2.018186,0.000005,2.037143,0.000017,0.018957
1306,2019-05-22 07:45:52.223098,4,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}",2.004512,0.000013,2.026882,0.000059,0.022370
1307,2019-05-22 07:45:53.465455,5,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}",2.001675,0.000009,2.025475,0.000045,0.023800
1308,2019-05-22 07:45:54.781749,6,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}",1.998562,0.000009,2.023556,0.000027,0.024994
1309,2019-05-22 07:45:55.984614,7,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}",1.994130,0.000001,2.020952,0.000053,0.026823


In [19]:
db_.commit()

In [20]:
len(selected_columns)

12

In [13]:
selected_columns = ['q25_roll_std_100',
 'abs_q25_5',
 'spkt_welch_density__coeff_3',
 'abs_q75_7',
 'spkt_welch_densitycoeff_2',
 "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
 'agg_autocorrelationf_agg_"mean"__maxlag_40',
 "number_peaks{'n': 1}",
 '5000peak_peak_amp_max_',
 'abs_q95_3',
 'spkt_welch_density__coeff_89',
 'abs_q05_2']

In [14]:
mytrial = []
#  tune hypterparameters
def objective(trial):
        
    max_depth = trial.suggest_int('max_depth', 2, 6)
    max_bin = trial.suggest_int('max_bin', 10, 100)
    eta = trial.suggest_uniform('eta', 0.01, 0.4)
    colsample_bytree = trial.suggest_uniform('colsample_bytree', 0.6, 1)
    min_child_weight = trial.suggest_int('min_child_weight', 200, 600)
    n_estimators = trial.suggest_int('n_estimators', 100, 500)
    subsample = trial.suggest_uniform('subsample', 0.6, 1.0)
    reg_lambda = trial.suggest_loguniform('reg_lambda', 0.000001, 1.0)
    reg_alpha = trial.suggest_loguniform('reg_alpha', 0.000001, 1.0)
    random_state = trial.suggest_int('random_state', 1, 9999)
        
    args={
        'columns':selected_columns,
        'kfold':{
            'n_splits': 8,
            'random_state': 1985,
            'shuffle': True,
            'type': 'group'
        },
        'scaler':{
            'cls':'StandardScaler',
            'init':{}
        },
        'algorithm':{
            'cls':'xgb.XGBRegressor',
            'init':{
                "max_depth":max_depth,
                "max_bin":max_bin,
                "eta":eta,
                "colsample_bytree":colsample_bytree,
                "min_child_weight":min_child_weight,
                "n_estimators":n_estimators,
                "subsample":subsample,
                "reg_lambda":reg_lambda,
                "reg_alpha":reg_alpha,
                'n_jobs':4
            },
            'fit':{
            },
        },
    }
    
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune 1444')
    val_mae_mean = np.mean(df_his.valid)
    val_mae_var = np.var(df_his.valid)
    train_mae_mean = np.mean(df_his.train)
    train_mae_var = np.var(df_his.train)
    
    trial.set_user_attr('val_mae', val_mae_mean)
    trial.set_user_attr('train_mae', train_mae_mean)
    trial.set_user_attr('mae_diff', val_mae_mean-train_mae_mean)
    trial.set_user_attr('val_mae_var', val_mae_var)

    return np.abs(val_mae_mean - train_mae_mean)*val_mae_mean

study = optuna.create_study()
study.optimize(objective, n_trials=200)

[I 2019-05-24 16:28:58,328] Finished trial#0 resulted in value: 0.5934772056258444. Current best value is 0.5934772056258444 with parameters: {'max_depth': 5, 'max_bin': 35, 'eta': 0.302492595967067, 'colsample_bytree': 0.7499497978323536, 'min_child_weight': 540, 'n_estimators': 412, 'subsample': 0.680900764437603, 'reg_lambda': 0.007004172066829176, 'reg_alpha': 2.4809985130629323e-05, 'random_state': 1308}.
[I 2019-05-24 16:29:53,512] Finished trial#1 resulted in value: 0.7215055709443015. Current best value is 0.5934772056258444 with parameters: {'max_depth': 5, 'max_bin': 35, 'eta': 0.302492595967067, 'colsample_bytree': 0.7499497978323536, 'min_child_weight': 540, 'n_estimators': 412, 'subsample': 0.680900764437603, 'reg_lambda': 0.007004172066829176, 'reg_alpha': 2.4809985130629323e-05, 'random_state': 1308}.
[I 2019-05-24 16:30:50,137] Finished trial#2 resulted in value: 0.4360132758749309. Current best value is 0.4360132758749309 with parameters: {'max_depth': 3, 'max_bin': 46

[I 2019-05-24 16:38:47,895] Finished trial#20 resulted in value: 0.3641821227859251. Current best value is 0.22356939745913645 with parameters: {'max_depth': 2, 'max_bin': 53, 'eta': 0.14451865810636566, 'colsample_bytree': 0.6363073522329966, 'min_child_weight': 301, 'n_estimators': 130, 'subsample': 0.7078030140393083, 'reg_lambda': 0.07029863781802004, 'reg_alpha': 2.256541515353986e-05, 'random_state': 7972}.
[I 2019-05-24 16:39:18,512] Finished trial#21 resulted in value: 0.3653867786907646. Current best value is 0.22356939745913645 with parameters: {'max_depth': 2, 'max_bin': 53, 'eta': 0.14451865810636566, 'colsample_bytree': 0.6363073522329966, 'min_child_weight': 301, 'n_estimators': 130, 'subsample': 0.7078030140393083, 'reg_lambda': 0.07029863781802004, 'reg_alpha': 2.256541515353986e-05, 'random_state': 7972}.
[I 2019-05-24 16:39:41,587] Finished trial#22 resulted in value: 0.2975856260849568. Current best value is 0.22356939745913645 with parameters: {'max_depth': 2, 'max_

[I 2019-05-24 16:50:13,435] Finished trial#40 resulted in value: 0.21223309384920433. Current best value is 0.20996181417979234 with parameters: {'max_depth': 2, 'max_bin': 62, 'eta': 0.04258175490723015, 'colsample_bytree': 0.7302089062588559, 'min_child_weight': 361, 'n_estimators': 103, 'subsample': 0.7091962532436572, 'reg_lambda': 0.02548598617040961, 'reg_alpha': 3.9261211676059685e-05, 'random_state': 9310}.
[I 2019-05-24 16:50:37,644] Finished trial#41 resulted in value: 0.32693914157240084. Current best value is 0.20996181417979234 with parameters: {'max_depth': 2, 'max_bin': 62, 'eta': 0.04258175490723015, 'colsample_bytree': 0.7302089062588559, 'min_child_weight': 361, 'n_estimators': 103, 'subsample': 0.7091962532436572, 'reg_lambda': 0.02548598617040961, 'reg_alpha': 3.9261211676059685e-05, 'random_state': 9310}.
[I 2019-05-24 16:50:57,570] Finished trial#42 resulted in value: 0.2620895054775156. Current best value is 0.20996181417979234 with parameters: {'max_depth': 2, '

[I 2019-05-24 17:00:47,026] Finished trial#59 resulted in value: 0.3556137744878315. Current best value is 0.20996181417979234 with parameters: {'max_depth': 2, 'max_bin': 62, 'eta': 0.04258175490723015, 'colsample_bytree': 0.7302089062588559, 'min_child_weight': 361, 'n_estimators': 103, 'subsample': 0.7091962532436572, 'reg_lambda': 0.02548598617040961, 'reg_alpha': 3.9261211676059685e-05, 'random_state': 9310}.
[I 2019-05-24 17:01:04,795] Finished trial#60 resulted in value: 0.2384221966883313. Current best value is 0.20996181417979234 with parameters: {'max_depth': 2, 'max_bin': 62, 'eta': 0.04258175490723015, 'colsample_bytree': 0.7302089062588559, 'min_child_weight': 361, 'n_estimators': 103, 'subsample': 0.7091962532436572, 'reg_lambda': 0.02548598617040961, 'reg_alpha': 3.9261211676059685e-05, 'random_state': 9310}.
[I 2019-05-24 17:02:17,359] Finished trial#61 resulted in value: 0.6064071120212207. Current best value is 0.20996181417979234 with parameters: {'max_depth': 2, 'ma

[I 2019-05-24 17:08:34,542] Finished trial#78 resulted in value: 0.3464152729006993. Current best value is 0.20996181417979234 with parameters: {'max_depth': 2, 'max_bin': 62, 'eta': 0.04258175490723015, 'colsample_bytree': 0.7302089062588559, 'min_child_weight': 361, 'n_estimators': 103, 'subsample': 0.7091962532436572, 'reg_lambda': 0.02548598617040961, 'reg_alpha': 3.9261211676059685e-05, 'random_state': 9310}.
[I 2019-05-24 17:09:05,510] Finished trial#79 resulted in value: 0.2958812741136576. Current best value is 0.20996181417979234 with parameters: {'max_depth': 2, 'max_bin': 62, 'eta': 0.04258175490723015, 'colsample_bytree': 0.7302089062588559, 'min_child_weight': 361, 'n_estimators': 103, 'subsample': 0.7091962532436572, 'reg_lambda': 0.02548598617040961, 'reg_alpha': 3.9261211676059685e-05, 'random_state': 9310}.
[I 2019-05-24 17:09:50,760] Finished trial#80 resulted in value: 0.47107536139333706. Current best value is 0.20996181417979234 with parameters: {'max_depth': 2, 'm

...

[I 2019-05-24 17:27:35,489] Finished trial#118 resulted in value: 0.2863974236172256. Current best value is 0.20531619256237935 with parameters: {'max_depth': 2, 'max_bin': 67, 'eta': 0.37866612951158884, 'colsample_bytree': 0.90075791557941, 'min_child_weight': 515, 'n_estimators': 102, 'subsample': 0.6970066706286289, 'reg_lambda': 0.0023765115729449883, 'reg_alpha': 0.07064884230693425, 'random_state': 7417}.
[I 2019-05-24 17:28:19,958] Finished trial#119 resulted in value: 0.43794144956959247. Current best value is 0.20531619256237935 with parameters: {'max_depth': 2, 'max_bin': 67, 'eta': 0.37866612951158884, 'colsample_bytree': 0.90075791557941, 'min_child_weight': 515, 'n_estimators': 102, 'subsample': 0.6970066706286289, 'reg_lambda': 0.0023765115729449883, 'reg_alpha': 0.07064884230693425, 'random_state': 7417}.
[I 2019-05-24 17:28:36,300] Finished trial#120 resulted in value: 0.2555156853360395. Current best value is 0.20531619256237935 with parameters: {'max_depth': 2, 'max_

[I 2019-05-24 17:37:39,230] Finished trial#138 resulted in value: 0.363506184164593. Current best value is 0.20531619256237935 with parameters: {'max_depth': 2, 'max_bin': 67, 'eta': 0.37866612951158884, 'colsample_bytree': 0.90075791557941, 'min_child_weight': 515, 'n_estimators': 102, 'subsample': 0.6970066706286289, 'reg_lambda': 0.0023765115729449883, 'reg_alpha': 0.07064884230693425, 'random_state': 7417}.
[I 2019-05-24 17:37:55,530] Finished trial#139 resulted in value: 0.28691830756347037. Current best value is 0.20531619256237935 with parameters: {'max_depth': 2, 'max_bin': 67, 'eta': 0.37866612951158884, 'colsample_bytree': 0.90075791557941, 'min_child_weight': 515, 'n_estimators': 102, 'subsample': 0.6970066706286289, 'reg_lambda': 0.0023765115729449883, 'reg_alpha': 0.07064884230693425, 'random_state': 7417}.
[I 2019-05-24 17:38:12,527] Finished trial#140 resulted in value: 0.24426729370934347. Current best value is 0.20531619256237935 with parameters: {'max_depth': 2, 'max_

[I 2019-05-24 17:43:06,676] Finished trial#158 resulted in value: 0.2315420068578455. Current best value is 0.2031799111523664 with parameters: {'max_depth': 2, 'max_bin': 92, 'eta': 0.3324280090268022, 'colsample_bytree': 0.8656290710895905, 'min_child_weight': 506, 'n_estimators': 100, 'subsample': 0.7737400669798493, 'reg_lambda': 3.342091389906669e-06, 'reg_alpha': 0.5561330490794749, 'random_state': 4782}.
[I 2019-05-24 17:43:24,753] Finished trial#159 resulted in value: 0.27783013281295355. Current best value is 0.2031799111523664 with parameters: {'max_depth': 2, 'max_bin': 92, 'eta': 0.3324280090268022, 'colsample_bytree': 0.8656290710895905, 'min_child_weight': 506, 'n_estimators': 100, 'subsample': 0.7737400669798493, 'reg_lambda': 3.342091389906669e-06, 'reg_alpha': 0.5561330490794749, 'random_state': 4782}.
[I 2019-05-24 17:43:42,851] Finished trial#160 resulted in value: 0.24872510213997884. Current best value is 0.2031799111523664 with parameters: {'max_depth': 2, 'max_bi

[I 2019-05-24 17:50:18,515] Finished trial#178 resulted in value: 0.3241406622902784. Current best value is 0.1981474525099577 with parameters: {'max_depth': 2, 'max_bin': 90, 'eta': 0.2918348555222015, 'colsample_bytree': 0.8496891740588636, 'min_child_weight': 541, 'n_estimators': 100, 'subsample': 0.6429441572710682, 'reg_lambda': 1.7331259220402156e-06, 'reg_alpha': 0.01042286378877082, 'random_state': 2990}.
[I 2019-05-24 17:50:43,529] Finished trial#179 resulted in value: 0.2567140540034464. Current best value is 0.1981474525099577 with parameters: {'max_depth': 2, 'max_bin': 90, 'eta': 0.2918348555222015, 'colsample_bytree': 0.8496891740588636, 'min_child_weight': 541, 'n_estimators': 100, 'subsample': 0.6429441572710682, 'reg_lambda': 1.7331259220402156e-06, 'reg_alpha': 0.01042286378877082, 'random_state': 2990}.
[I 2019-05-24 17:51:16,008] Finished trial#180 resulted in value: 0.35930298676318295. Current best value is 0.1981474525099577 with parameters: {'max_depth': 2, 'max

[I 2019-05-24 18:01:17,182] Finished trial#198 resulted in value: 0.21300730021438605. Current best value is 0.1981474525099577 with parameters: {'max_depth': 2, 'max_bin': 90, 'eta': 0.2918348555222015, 'colsample_bytree': 0.8496891740588636, 'min_child_weight': 541, 'n_estimators': 100, 'subsample': 0.6429441572710682, 'reg_lambda': 1.7331259220402156e-06, 'reg_alpha': 0.01042286378877082, 'random_state': 2990}.
[I 2019-05-24 18:03:02,006] Finished trial#199 resulted in value: 0.6722063683965686. Current best value is 0.1981474525099577 with parameters: {'max_depth': 2, 'max_bin': 90, 'eta': 0.2918348555222015, 'colsample_bytree': 0.8496891740588636, 'min_child_weight': 541, 'n_estimators': 100, 'subsample': 0.6429441572710682, 'reg_lambda': 1.7331259220402156e-06, 'reg_alpha': 0.01042286378877082, 'random_state': 2990}.


In [15]:
for trial_i in mytrial:
    db.insert(trial_i)

In [30]:
df_trial = db.select()
df_trial[(df_trial['remark']=='tune 1444')&(df_trial['mae_diff']<.1)][['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']].sort_values(by=['val_mae'])

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
1591,2019-05-24 17:40:13.932552,tune 1444,12,1.982186,0.012094,2.081635,0.536821,0.099449
1595,2019-05-24 17:41:00.111691,tune 1444,12,1.982602,0.012136,2.082274,0.537450,0.099673
1598,2019-05-24 17:41:35.382846,tune 1444,12,1.986369,0.012140,2.083870,0.543122,0.097501
1593,2019-05-24 17:40:37.671265,tune 1444,12,1.986384,0.012199,2.083987,0.541126,0.097603
1616,2019-05-24 17:46:26.830548,tune 1444,12,1.989262,0.012382,2.084328,0.537684,0.095065
1536,2019-05-24 17:13:33.245768,tune 1444,12,1.986799,0.012208,2.085260,0.539925,0.098461
1545,2019-05-24 17:18:38.683974,tune 1444,12,1.986982,0.012146,2.085623,0.536669,0.098641


In [19]:
db.commit()

In [20]:
mytrial = []
#  tune hypterparameters
def objective(trial):
        
    max_depth = trial.suggest_int('max_depth', 2, 6)
    max_bin = trial.suggest_int('max_bin', 10, 100)
    eta = trial.suggest_uniform('eta', 0.01, 0.4)
    colsample_bytree = trial.suggest_uniform('colsample_bytree', 0.6, 1)
    min_child_weight = trial.suggest_int('min_child_weight', 200, 600)
    n_estimators = trial.suggest_int('n_estimators', 100, 500)
    subsample = trial.suggest_uniform('subsample', 0.6, 1.0)
    reg_lambda = trial.suggest_loguniform('reg_lambda', 0.000001, 1.0)
    reg_alpha = trial.suggest_loguniform('reg_alpha', 0.000001, 1.0)
    random_state = trial.suggest_int('random_state', 1, 9999)
        
    args={
        'columns':selected_columns,
        'kfold':{
            'n_splits': 3,
            'random_state': 1985,
            'shuffle': True,
            'type': 'stratified'
        },
        'scaler':{
            'cls':'StandardScaler',
            'init':{}
        },
        'algorithm':{
            'cls':'xgb.XGBRegressor',
            'init':{
                "max_depth":max_depth,
                "max_bin":max_bin,
                "eta":eta,
                "colsample_bytree":colsample_bytree,
                "min_child_weight":min_child_weight,
                "n_estimators":n_estimators,
                "subsample":subsample,
                "reg_lambda":reg_lambda,
                "reg_alpha":reg_alpha,
                'n_jobs':4
            },
            'fit':{
            },
        },
    }
    
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune 1444 by stratified')
    val_mae_mean = np.mean(df_his.valid)
    val_mae_var = np.var(df_his.valid)
    train_mae_mean = np.mean(df_his.train)
    train_mae_var = np.var(df_his.train)
    
    trial.set_user_attr('val_mae', val_mae_mean)
    trial.set_user_attr('train_mae', train_mae_mean)
    trial.set_user_attr('mae_diff', val_mae_mean-train_mae_mean)
    trial.set_user_attr('val_mae_var', val_mae_var)

    return np.abs(val_mae_mean - train_mae_mean)*val_mae_mean

study = optuna.create_study()
study.optimize(objective, n_trials=200)

[I 2019-05-24 18:07:58,115] Finished trial#0 resulted in value: 0.09654915226932492. Current best value is 0.09654915226932492 with parameters: {'max_depth': 2, 'max_bin': 74, 'eta': 0.3575927818609957, 'colsample_bytree': 0.9314810993170525, 'min_child_weight': 255, 'n_estimators': 463, 'subsample': 0.9725118873482428, 'reg_lambda': 0.11126424611801736, 'reg_alpha': 5.179310231653595e-06, 'random_state': 7733}.
[I 2019-05-24 18:08:19,722] Finished trial#1 resulted in value: 0.19048093555966372. Current best value is 0.09654915226932492 with parameters: {'max_depth': 2, 'max_bin': 74, 'eta': 0.3575927818609957, 'colsample_bytree': 0.9314810993170525, 'min_child_weight': 255, 'n_estimators': 463, 'subsample': 0.9725118873482428, 'reg_lambda': 0.11126424611801736, 'reg_alpha': 5.179310231653595e-06, 'random_state': 7733}.
[I 2019-05-24 18:08:28,902] Finished trial#2 resulted in value: 0.0825189805910531. Current best value is 0.0825189805910531 with parameters: {'max_depth': 3, 'max_bin'

[I 2019-05-24 18:10:53,902] Finished trial#19 resulted in value: 0.04243787830732163. Current best value is 0.04164538312850546 with parameters: {'max_depth': 2, 'max_bin': 79, 'eta': 0.055115526030071034, 'colsample_bytree': 0.6703701999250644, 'min_child_weight': 546, 'n_estimators': 173, 'subsample': 0.6421071261557975, 'reg_lambda': 0.00017170206803463595, 'reg_alpha': 9.992062468315778e-06, 'random_state': 2174}.
[I 2019-05-24 18:11:02,040] Finished trial#20 resulted in value: 0.05533363543695176. Current best value is 0.04164538312850546 with parameters: {'max_depth': 2, 'max_bin': 79, 'eta': 0.055115526030071034, 'colsample_bytree': 0.6703701999250644, 'min_child_weight': 546, 'n_estimators': 173, 'subsample': 0.6421071261557975, 'reg_lambda': 0.00017170206803463595, 'reg_alpha': 9.992062468315778e-06, 'random_state': 2174}.
[I 2019-05-24 18:11:09,490] Finished trial#21 resulted in value: 0.08313694765828131. Current best value is 0.04164538312850546 with parameters: {'max_depth

[I 2019-05-24 18:13:51,736] Finished trial#38 resulted in value: 0.10833673226807883. Current best value is 0.04015266006490325 with parameters: {'max_depth': 2, 'max_bin': 100, 'eta': 0.12208002424566186, 'colsample_bytree': 0.7574976142839639, 'min_child_weight': 561, 'n_estimators': 160, 'subsample': 0.6897189341673549, 'reg_lambda': 2.0284672552003527e-05, 'reg_alpha': 0.00014611760060092763, 'random_state': 139}.
[I 2019-05-24 18:14:01,843] Finished trial#39 resulted in value: 0.12350300993173415. Current best value is 0.04015266006490325 with parameters: {'max_depth': 2, 'max_bin': 100, 'eta': 0.12208002424566186, 'colsample_bytree': 0.7574976142839639, 'min_child_weight': 561, 'n_estimators': 160, 'subsample': 0.6897189341673549, 'reg_lambda': 2.0284672552003527e-05, 'reg_alpha': 0.00014611760060092763, 'random_state': 139}.
[I 2019-05-24 18:14:06,691] Finished trial#40 resulted in value: 0.05363298275198379. Current best value is 0.04015266006490325 with parameters: {'max_depth

[I 2019-05-24 18:17:12,947] Finished trial#57 resulted in value: 0.052765888691753825. Current best value is 0.03669089845194971 with parameters: {'max_depth': 2, 'max_bin': 56, 'eta': 0.3126756353868539, 'colsample_bytree': 0.8693108907279761, 'min_child_weight': 277, 'n_estimators': 100, 'subsample': 0.9277174698987204, 'reg_lambda': 0.00017365837666918308, 'reg_alpha': 5.727717460573909e-05, 'random_state': 2795}.
[I 2019-05-24 18:17:19,265] Finished trial#58 resulted in value: 0.09149110241765802. Current best value is 0.03669089845194971 with parameters: {'max_depth': 2, 'max_bin': 56, 'eta': 0.3126756353868539, 'colsample_bytree': 0.8693108907279761, 'min_child_weight': 277, 'n_estimators': 100, 'subsample': 0.9277174698987204, 'reg_lambda': 0.00017365837666918308, 'reg_alpha': 5.727717460573909e-05, 'random_state': 2795}.
[I 2019-05-24 18:17:23,108] Finished trial#59 resulted in value: 0.04430596917934166. Current best value is 0.03669089845194971 with parameters: {'max_depth': 

[I 2019-05-24 18:19:29,468] Finished trial#76 resulted in value: 0.03543505962904276. Current best value is 0.03215300861904 with parameters: {'max_depth': 2, 'max_bin': 75, 'eta': 0.21479896890932804, 'colsample_bytree': 0.8567357676247367, 'min_child_weight': 410, 'n_estimators': 100, 'subsample': 0.7222196829638388, 'reg_lambda': 3.65148836718332e-05, 'reg_alpha': 1.104799716997286e-05, 'random_state': 2898}.
[I 2019-05-24 18:19:32,460] Finished trial#77 resulted in value: 0.03613329603232274. Current best value is 0.03215300861904 with parameters: {'max_depth': 2, 'max_bin': 75, 'eta': 0.21479896890932804, 'colsample_bytree': 0.8567357676247367, 'min_child_weight': 410, 'n_estimators': 100, 'subsample': 0.7222196829638388, 'reg_lambda': 3.65148836718332e-05, 'reg_alpha': 1.104799716997286e-05, 'random_state': 2898}.
[I 2019-05-24 18:19:36,511] Finished trial#78 resulted in value: 0.06142428236208545. Current best value is 0.03215300861904 with parameters: {'max_depth': 2, 'max_bin'

...

[I 2019-05-24 18:24:55,001] Finished trial#116 resulted in value: 0.06343236704509365. Current best value is 0.03215300861904 with parameters: {'max_depth': 2, 'max_bin': 75, 'eta': 0.21479896890932804, 'colsample_bytree': 0.8567357676247367, 'min_child_weight': 410, 'n_estimators': 100, 'subsample': 0.7222196829638388, 'reg_lambda': 3.65148836718332e-05, 'reg_alpha': 1.104799716997286e-05, 'random_state': 2898}.
[I 2019-05-24 18:24:57,815] Finished trial#117 resulted in value: 0.036539723687045694. Current best value is 0.03215300861904 with parameters: {'max_depth': 2, 'max_bin': 75, 'eta': 0.21479896890932804, 'colsample_bytree': 0.8567357676247367, 'min_child_weight': 410, 'n_estimators': 100, 'subsample': 0.7222196829638388, 'reg_lambda': 3.65148836718332e-05, 'reg_alpha': 1.104799716997286e-05, 'random_state': 2898}.
[I 2019-05-24 18:25:02,948] Finished trial#118 resulted in value: 0.05011605699506957. Current best value is 0.03215300861904 with parameters: {'max_depth': 2, 'max_

[I 2019-05-24 18:27:30,874] Finished trial#135 resulted in value: 0.093931845150868. Current best value is 0.030217154784635276 with parameters: {'max_depth': 2, 'max_bin': 95, 'eta': 0.12476969025852838, 'colsample_bytree': 0.9776056476393447, 'min_child_weight': 539, 'n_estimators': 111, 'subsample': 0.6192233800189538, 'reg_lambda': 0.0003048989928800843, 'reg_alpha': 2.8307917457949566e-06, 'random_state': 1069}.
[I 2019-05-24 18:27:40,560] Finished trial#136 resulted in value: 0.10231570070205151. Current best value is 0.030217154784635276 with parameters: {'max_depth': 2, 'max_bin': 95, 'eta': 0.12476969025852838, 'colsample_bytree': 0.9776056476393447, 'min_child_weight': 539, 'n_estimators': 111, 'subsample': 0.6192233800189538, 'reg_lambda': 0.0003048989928800843, 'reg_alpha': 2.8307917457949566e-06, 'random_state': 1069}.
[I 2019-05-24 18:27:46,232] Finished trial#137 resulted in value: 0.060124083803343394. Current best value is 0.030217154784635276 with parameters: {'max_de

[I 2019-05-24 18:29:19,875] Finished trial#154 resulted in value: 0.03454987886449087. Current best value is 0.02970588008209703 with parameters: {'max_depth': 2, 'max_bin': 98, 'eta': 0.13247579018677344, 'colsample_bytree': 0.910822981876779, 'min_child_weight': 518, 'n_estimators': 100, 'subsample': 0.6800979348206516, 'reg_lambda': 1.088991996638132e-06, 'reg_alpha': 0.038003709960088644, 'random_state': 1347}.
[I 2019-05-24 18:29:26,468] Finished trial#155 resulted in value: 0.06209968631253818. Current best value is 0.02970588008209703 with parameters: {'max_depth': 2, 'max_bin': 98, 'eta': 0.13247579018677344, 'colsample_bytree': 0.910822981876779, 'min_child_weight': 518, 'n_estimators': 100, 'subsample': 0.6800979348206516, 'reg_lambda': 1.088991996638132e-06, 'reg_alpha': 0.038003709960088644, 'random_state': 1347}.
[I 2019-05-24 18:29:29,569] Finished trial#156 resulted in value: 0.02805787112781474. Current best value is 0.02805787112781474 with parameters: {'max_depth': 2,

[I 2019-05-24 18:31:51,718] Finished trial#174 resulted in value: 0.057035717057714326. Current best value is 0.02805787112781474 with parameters: {'max_depth': 2, 'max_bin': 89, 'eta': 0.03920011747338854, 'colsample_bytree': 0.922180110327549, 'min_child_weight': 543, 'n_estimators': 100, 'subsample': 0.6241314051606546, 'reg_lambda': 1.0237667772951773e-05, 'reg_alpha': 0.9304736908819962, 'random_state': 458}.
[I 2019-05-24 18:31:56,083] Finished trial#175 resulted in value: 0.04762313875465133. Current best value is 0.02805787112781474 with parameters: {'max_depth': 2, 'max_bin': 89, 'eta': 0.03920011747338854, 'colsample_bytree': 0.922180110327549, 'min_child_weight': 543, 'n_estimators': 100, 'subsample': 0.6241314051606546, 'reg_lambda': 1.0237667772951773e-05, 'reg_alpha': 0.9304736908819962, 'random_state': 458}.
[I 2019-05-24 18:32:04,709] Finished trial#176 resulted in value: 0.05980916482909658. Current best value is 0.02805787112781474 with parameters: {'max_depth': 2, 'm

[I 2019-05-24 18:33:56,313] Finished trial#194 resulted in value: 0.04789263553739756. Current best value is 0.02805787112781474 with parameters: {'max_depth': 2, 'max_bin': 89, 'eta': 0.03920011747338854, 'colsample_bytree': 0.922180110327549, 'min_child_weight': 543, 'n_estimators': 100, 'subsample': 0.6241314051606546, 'reg_lambda': 1.0237667772951773e-05, 'reg_alpha': 0.9304736908819962, 'random_state': 458}.
[I 2019-05-24 18:34:06,547] Finished trial#195 resulted in value: 0.08583071781683436. Current best value is 0.02805787112781474 with parameters: {'max_depth': 2, 'max_bin': 89, 'eta': 0.03920011747338854, 'colsample_bytree': 0.922180110327549, 'min_child_weight': 543, 'n_estimators': 100, 'subsample': 0.6241314051606546, 'reg_lambda': 1.0237667772951773e-05, 'reg_alpha': 0.9304736908819962, 'random_state': 458}.
[I 2019-05-24 18:34:09,990] Finished trial#196 resulted in value: 0.036284938348048225. Current best value is 0.02805787112781474 with parameters: {'max_depth': 2, 'm

In [21]:
for trial_i in mytrial:
    db.insert(trial_i)

In [27]:
df_trial = db.select()
df_trial[(df_trial['remark']=='tune 1444 by stratified')&(df_trial['mae_diff']<.05)][['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']].sort_values(by=['val_mae'])

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
1646,2019-05-24 18:07:58.010608,tune 1444 by stratified,12,1.892733,0.000018,1.942439,0.000084,0.049705
1781,2019-05-24 18:27:30.754147,tune 1444 by stratified,12,1.897909,0.000026,1.946174,0.000098,0.048265
1659,2019-05-24 18:10:09.885777,tune 1444 by stratified,12,1.897719,0.000016,1.946963,0.000162,0.049245
1689,2019-05-24 18:14:38.247460,tune 1444 by stratified,12,1.898627,0.000012,1.947239,0.000083,0.048612
1721,2019-05-24 18:19:26.337447,tune 1444 by stratified,12,1.900512,0.000039,1.947267,0.000093,0.046755
1790,2019-05-24 18:28:44.545773,tune 1444 by stratified,12,1.899469,0.000012,1.947416,0.000078,0.047947
1753,2019-05-24 18:23:33.811422,tune 1444 by stratified,12,1.902664,0.000019,1.947674,0.000149,0.045009
1746,2019-05-24 18:22:37.559336,tune 1444 by stratified,12,1.901846,0.000019,1.947704,0.000049,0.045858
1698,2019-05-24 18:16:31.372664,tune 1444 by stratified,12,1.899234,0.000006,1.947899,0.000114,0.048664
1671,2019-05-24 18:11:31.337186,tune 1444 by stratified,12,1.902978,0.000022,1.948845,0.000102,0.045867


In [24]:
db.commit()